In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
data_to_split = pd.read_csv('train_val_metadata.csv')

In [7]:
number_of_folds = 5
folds = []

In [9]:
len(data_to_split)

5565

In [11]:
data_to_split.columns.values.tolist()

['patient id',
 'source',
 'country',
 'sex',
 'age',
 'finding',
 'verified finding',
 'slice selection',
 'view',
 'modality']

In [129]:
# findig the images for the patients

In [13]:
patient_images = pd.read_csv('txt/train_val_COVIDx_CT-3A.csv', sep=' ',header=0)
full_set = pd.DataFrame(columns=data_to_split.columns.values.tolist())
#full_set = data_to_split
data_to_split['images'] = ' '
full_set['images'] = ' '

In [15]:
for i, row in data_to_split.iterrows():
    mask = patient_images['fname'].str.match(pat='{}[^A-Za-z0-9]'.format(row['patient id']))
    images = np.array(patient_images[mask])
    if images.size == 0:
        continue
    else:
        full_set.loc[i] = row
        full_set.at[i,'images'] = images[:,0]

In [17]:
full_set.reset_index(drop=True, inplace=True)

In [19]:
full_set

,patient id,source,country,sex,age,finding,verified finding,slice selection,view,modality,images
0,CP_10,CNCB,China,NaN,NaN,Pneumonia,Yes,Automatic,Axial,CT,"[CP_10_3153_0172.png, CP_10_3153_0175.png, CP_..."
1,CP_1068,CNCB,China,NaN,NaN,Pneumonia,Yes,Automatic,Axial,CT,CP_1068_3107_0021.png
2,CP_1071,CNCB,China,NaN,NaN,Pneumonia,Yes,Expert,Axial,CT,"[CP_1071_3114_0035.png, CP_1071_3114_0036.png,..."
3,CP_1072,CNCB,China,NaN,NaN,Pneumonia,Yes,Expert,Axial,CT,"[CP_1072_3115_0037.png, CP_1072_3115_0038.png,..."
4,CP_1073,CNCB,China,NaN,NaN,Pneumonia,Yes,Expert,Axial,CT,"[CP_1073_3116_0031.png, CP_1073_3116_0033.png,..."
...,...,...,...,...,...,...,...,...,...,...,...
4507,COVIDCTMD-normal071,COVID-CT-MD,Iran,F,43.0,Normal,Yes,NaN,Axial,CT,"[COVIDCTMD-normal071-IM0001.png, COVIDCTMD-nor..."
4508,COVIDCTMD-normal072,COVID-CT-MD,Iran,M,31.0,Normal,Yes,NaN,Axial,CT,"[COVIDCTMD-normal072-IM0001.png, COVIDCTMD-nor..."
4509,COVIDCTMD-normal073,COVID-CT-MD,Iran,M,39.0,Normal,Yes,NaN,Axial,CT,"[COVIDCTMD-normal073-IM0001.png, COVIDCTMD-nor..."
4510,COVIDCTMD-normal074,COVID-CT-MD,Iran,M,40.0,Normal,Yes,NaN,Axial,CT,"[COVIDCTMD-normal074-IM0001.png, COVIDCTMD-nor..."


In [51]:
image_count = 0
for i in range(0, len(full_set)):
    image_count = image_count + full_set['images'][i].size

In [53]:
image_count

381596

In [25]:
full_set.iloc[0]['images']

array(['CP_10_3153_0172.png', 'CP_10_3153_0175.png',
       'CP_10_3153_0182.png', 'CP_10_3153_0183.png',
       'CP_10_3153_0191.png'], dtype=object)

In [55]:
already_sampled = pd.DataFrame(columns=full_set.columns.values.tolist())
images_remaining = image_count
for i in range(0,number_of_folds):
    print("Now at fold: " + str(i))
    print("Already sampled: " + str(len(already_sampled)))
    print("Images remaining: " + str(images_remaining))
    training_set = pd.DataFrame(columns=full_set.columns.values.tolist())
    val_set = pd.DataFrame(columns=full_set.columns.values.tolist())
    
    training_set = full_set

    j = 0
    
    while(j < int(len(full_set)/number_of_folds)):
        sample = training_set.sample(n=1)
        if sample['patient id'].isin(already_sampled['patient id']).values[0]:
            continue
        else:
            val_set = pd.concat([val_set,sample])
            already_sampled = pd.concat([already_sampled,sample])
            training_set = training_set.drop(sample.index, inplace=False)
            images_remaining = images_remaining - sample.iloc[0]['images'].size
            j = j + 1

    
    temp = []
    temp.append(training_set)
    temp.append(val_set)
    folds.append(temp)

Now at fold: 0
Already sampled: 0
Images remaining: 381596
Now at fold: 1
Already sampled: 902
Images remaining: 306031
Now at fold: 2
Already sampled: 1804
Images remaining: 229754
Now at fold: 3
Already sampled: 2706
Images remaining: 152867
Now at fold: 4
Already sampled: 3608
Images remaining: 76312


In [57]:
images_remaining

343

In [39]:
len(folds[4][1])

902

In [41]:
len(already_sampled)

4510

In [43]:
for i in range(0, len(folds)):
    print("Training set length: " + str(len(folds[i][0])))
    print("Validation set length: " + str(len(folds[i][1])))
    print("Total dataset length: " + str(len(folds[i][0]) + len(folds[i][1])))
    print("\n")

Training set length: 3610
Validation set length: 902
Total dataset length: 4512


Training set length: 3610
Validation set length: 902
Total dataset length: 4512


Training set length: 3610
Validation set length: 902
Total dataset length: 4512


Training set length: 3610
Validation set length: 902
Total dataset length: 4512


Training set length: 3610
Validation set length: 902
Total dataset length: 4512




In [45]:
# Distribution of class labels in training data
for i in range(0, len(folds)):
    print(folds[i][0]['finding'].value_counts(normalize=True))

finding
COVID-19     0.768144
Pneumonia    0.172022
Normal       0.059834
Name: proportion, dtype: float64
finding
COVID-19     0.760942
Pneumonia    0.175069
Normal       0.063989
Name: proportion, dtype: float64
finding
COVID-19     0.763158
Pneumonia    0.177562
Normal       0.059280
Name: proportion, dtype: float64
finding
COVID-19     0.768975
Pneumonia    0.176177
Normal       0.054848
Name: proportion, dtype: float64
finding
COVID-19     0.760942
Pneumonia    0.178947
Normal       0.060111
Name: proportion, dtype: float64


In [47]:
# Distribution of class labels in validation data
for i in range(0, len(folds)):
    print(folds[i][1]['finding'].value_counts(normalize=True))

finding
COVID-19     0.749446
Pneumonia    0.191796
Normal       0.058758
Name: proportion, dtype: float64
finding
COVID-19     0.778271
Pneumonia    0.179601
Normal       0.042129
Name: proportion, dtype: float64
finding
COVID-19     0.769401
Pneumonia    0.169623
Normal       0.060976
Name: proportion, dtype: float64
finding
COVID-19     0.746120
Pneumonia    0.175166
Normal       0.078714
Name: proportion, dtype: float64
finding
COVID-19     0.778271
Pneumonia    0.164080
Normal       0.057650
Name: proportion, dtype: float64


In [49]:
for i in range(0, len(folds)):
    print(folds[4][1]['patient id'].reset_index(drop=True) == folds[i][1]['patient id'].reset_index(drop=True))

0      False
1      False
2      False
3      False
4      False
       ...  
897    False
898    False
899    False
900    False
901    False
Name: patient id, Length: 902, dtype: bool
0      False
1      False
2      False
3      False
4      False
       ...  
897    False
898    False
899    False
900    False
901    False
Name: patient id, Length: 902, dtype: bool
0      False
1      False
2      False
3      False
4      False
       ...  
897    False
898    False
899    False
900    False
901    False
Name: patient id, Length: 902, dtype: bool
0      False
1      False
2      False
3      False
4      False
       ...  
897    False
898    False
899    False
900    False
901    False
Name: patient id, Length: 902, dtype: bool
0      True
1      True
2      True
3      True
4      True
       ... 
897    True
898    True
899    True
900    True
901    True
Name: patient id, Length: 902, dtype: bool


In [133]:
from sklearn.model_selection import KFold

In [135]:
kf = KFold(n_splits=number_of_folds, shuffle=True, random_state=11)

In [137]:
folds_kfolds = []

In [139]:
for i, (train_index, val_index) in enumerate(kf.split(full_set)):
    split = []
    split.append(full_set.iloc[train_index])
    split.append(full_set.iloc[val_index])
    folds_kfolds.append(split)

In [141]:
for i in range(0, len(folds_kfolds)):
    print("Training set length: " + str(len(folds_kfolds[i][0])))
    print("Validation set length: " + str(len(folds_kfolds[i][1])))
    print("Total dataset length: " + str(len(folds_kfolds[i][0]) + len(folds_kfolds[i][1])))
    print("\n")

Training set length: 4452
Validation set length: 1113
Total dataset length: 5565


Training set length: 4452
Validation set length: 1113
Total dataset length: 5565


Training set length: 4452
Validation set length: 1113
Total dataset length: 5565


Training set length: 4452
Validation set length: 1113
Total dataset length: 5565


Training set length: 4452
Validation set length: 1113
Total dataset length: 5565




In [143]:
# Distribution of class labels in training data
for i in range(0, len(folds_kfolds)):
    print(folds_kfolds[i][0]['finding'].value_counts(normalize=True))

finding
COVID-19     0.772237
Pneumonia    0.140836
Normal       0.086927
Name: proportion, dtype: float64
finding
COVID-19     0.772013
Pneumonia    0.140162
Normal       0.087826
Name: proportion, dtype: float64
finding
COVID-19     0.766622
Pneumonia    0.148248
Normal       0.085130
Name: proportion, dtype: float64
finding
COVID-19     0.769766
Pneumonia    0.142183
Normal       0.088050
Name: proportion, dtype: float64
finding
COVID-19     0.770216
Pneumonia    0.141959
Normal       0.087826
Name: proportion, dtype: float64


In [145]:
# Distribution of class labels in validation data
for i in range(0, len(folds_kfolds)):
    print(folds_kfolds[i][1]['finding'].value_counts(normalize=True))

finding
COVID-19     0.761905
Pneumonia    0.150045
Normal       0.088050
Name: proportion, dtype: float64
finding
COVID-19     0.762803
Pneumonia    0.152740
Normal       0.084456
Name: proportion, dtype: float64
finding
COVID-19     0.784367
Pneumonia    0.120395
Normal       0.095238
Name: proportion, dtype: float64
finding
COVID-19     0.771788
Pneumonia    0.144654
Normal       0.083558
Name: proportion, dtype: float64
finding
COVID-19     0.769991
Pneumonia    0.145553
Normal       0.084456
Name: proportion, dtype: float64


In [147]:
from pandas.testing import assert_frame_equal

In [149]:
for i in range(0, len(folds_kfolds)):
    print(folds_kfolds[0][1]['patient id'].reset_index(drop=True) == folds_kfolds[i][1]['patient id'].reset_index(drop=True))

0       True
1       True
2       True
3       True
4       True
        ... 
1108    True
1109    True
1110    True
1111    True
1112    True
Name: patient id, Length: 1113, dtype: bool
0       False
1       False
2       False
3       False
4       False
        ...  
1108    False
1109    False
1110    False
1111    False
1112    False
Name: patient id, Length: 1113, dtype: bool
0       False
1       False
2       False
3       False
4       False
        ...  
1108    False
1109    False
1110    False
1111    False
1112    False
Name: patient id, Length: 1113, dtype: bool
0       False
1       False
2       False
3       False
4       False
        ...  
1108    False
1109    False
1110    False
1111    False
1112    False
Name: patient id, Length: 1113, dtype: bool
0       False
1       False
2       False
3       False
4       False
        ...  
1108    False
1109    False
1110    False
1111    False
1112    False
Name: patient id, Length: 1113, dtype: bool


In [155]:
folds[0][0]

,patient id,source,country,sex,age,finding,verified finding,slice selection,view,modality,images
0,CP_10,CNCB,China,NaN,NaN,Pneumonia,Yes,Automatic,Axial,CT,"[[CP_10_3153_0172.png, 1, 30, 93, 474, 419], [..."
1,CP_1068,CNCB,China,NaN,NaN,Pneumonia,Yes,Automatic,Axial,CT,"[[CP_1068_3107_0021.png, 1, 0, 113, 512, 388]]"
2,CP_1071,CNCB,China,NaN,NaN,Pneumonia,Yes,Expert,Axial,CT,"[[CP_1071_3114_0035.png, 1, 33, 117, 501, 451]..."
3,CP_1072,CNCB,China,NaN,NaN,Pneumonia,Yes,Expert,Axial,CT,"[[CP_1072_3115_0037.png, 1, 46, 73, 472, 385],..."
4,CP_1073,CNCB,China,NaN,NaN,Pneumonia,Yes,Expert,Axial,CT,"[[CP_1073_3116_0031.png, 1, 22, 71, 503, 389],..."
...,...,...,...,...,...,...,...,...,...,...,...
5557,COVIDCTMD-normal066,COVID-CT-MD,Iran,M,30.0,Normal,Yes,NaN,Axial,CT,"[[COVIDCTMD-normal066-IM0001.png, 0, 0, 159, 5..."
5558,COVIDCTMD-normal069,COVID-CT-MD,Iran,F,26.0,Normal,Yes,NaN,Axial,CT,"[[COVIDCTMD-normal069-IM0001.png, 0, 0, 186, 5..."
5559,COVIDCTMD-normal070,COVID-CT-MD,Iran,M,60.0,Normal,Yes,NaN,Axial,CT,"[[COVIDCTMD-normal070-IM0001.png, 0, 0, 161, 5..."
5563,COVIDCTMD-normal074,COVID-CT-MD,Iran,M,40.0,Normal,Yes,NaN,Axial,CT,"[[COVIDCTMD-normal074-IM0001.png, 0, 0, 126, 5..."


In [157]:
folds_kfolds[0][0]

,patient id,source,country,sex,age,finding,verified finding,slice selection,view,modality,images
0,CP_10,CNCB,China,NaN,NaN,Pneumonia,Yes,Automatic,Axial,CT,"[[CP_10_3153_0172.png, 1, 30, 93, 474, 419], [..."
2,CP_1071,CNCB,China,NaN,NaN,Pneumonia,Yes,Expert,Axial,CT,"[[CP_1071_3114_0035.png, 1, 33, 117, 501, 451]..."
3,CP_1072,CNCB,China,NaN,NaN,Pneumonia,Yes,Expert,Axial,CT,"[[CP_1072_3115_0037.png, 1, 46, 73, 472, 385],..."
4,CP_1073,CNCB,China,NaN,NaN,Pneumonia,Yes,Expert,Axial,CT,"[[CP_1073_3116_0031.png, 1, 22, 71, 503, 389],..."
6,CP_1076,CNCB,China,NaN,NaN,Pneumonia,Yes,Expert,Axial,CT,"[[CP_1076_3120_0045.png, 1, 57, 84, 468, 396],..."
...,...,...,...,...,...,...,...,...,...,...,...
5560,COVIDCTMD-normal071,COVID-CT-MD,Iran,F,43.0,Normal,Yes,NaN,Axial,CT,"[[COVIDCTMD-normal071-IM0001.png, 0, 0, 97, 51..."
5561,COVIDCTMD-normal072,COVID-CT-MD,Iran,M,31.0,Normal,Yes,NaN,Axial,CT,"[[COVIDCTMD-normal072-IM0001.png, 0, 0, 139, 5..."
5562,COVIDCTMD-normal073,COVID-CT-MD,Iran,M,39.0,Normal,Yes,NaN,Axial,CT,"[[COVIDCTMD-normal073-IM0001.png, 0, 0, 111, 5..."
5563,COVIDCTMD-normal074,COVID-CT-MD,Iran,M,40.0,Normal,Yes,NaN,Axial,CT,"[[COVIDCTMD-normal074-IM0001.png, 0, 0, 126, 5..."
